In [1]:
import pandas as pd
import numpy as np
import sqlite3
import math

from numpy import transpose
from numpy.linalg import inv,pinv, det
from scipy.stats import norm

engine = sqlite3.connect('yahoo')
np.set_printoptions(precision=3)
click_cols = ['time', 'user_1', 'user_2', 'user_3', 'user_4', 'user_5', 'user_6', 'displayed', 'clicked']

In [2]:
articles_df = pd.read_sql_query('SELECT * FROM articles',con=engine).set_index('index')
clicks_df = pd.read_sql_query('''SELECT * FROM clicks WHERE (ABS(CAST((RANDOM()) as int)) % 100) < 5''', con=engine)
clicks_df[click_cols] = clicks_df[click_cols].apply(pd.to_numeric, ) # Temporary workaround, this should be done before sql

In [3]:
#articles_df.head()

In [4]:
#clicks_df.shape

In [5]:
#clicks_df.head()

In [6]:
"""
Evaluation Protocol
===================
Let σ be an input stream of logged events to be used for the simulation,
where an event consists of a tuple consisting of the user vector, 
20 article vectors (context vectors), the displayed vector (selected arm), 
and the result (observed reward).
Step through each element s sampled from the stream.
Let t denote the current time-step in the algorithm and h_{t-1} prior history.
If given h_{t-1} and the current context, the algorithm picks the same vector
as the selected arm - retain the event and add it to the history. Otherwise,
keep processing items.
"""

'\nEvaluation Protocol\n===================\nLet σ be an input stream of logged events to be used for the simulation,\nwhere an event consists of a tuple consisting of the user vector, \n20 article vectors (context vectors), the displayed vector (selected arm), \nand the result (observed reward).\nStep through each element s sampled from the stream.\nLet t denote the current time-step in the algorithm and h_{t-1} prior history.\nIf given h_{t-1} and the current context, the algorithm picks the same vector\nas the selected arm - retain the event and add it to the history. Otherwise,\nkeep processing items.\n'

In [7]:
def eta(T):
    """ 
    Generates the cutoff probabilities for exploration rounds in interval chaining. 
    
    :param T: the total number of iterations
    """
    return np.array([pow(t, -1/3) for t in range(1,T+1)])

In [8]:
user_feat_mat = clicks_df.as_matrix(['user_1', 'user_2', 'user_3', 'user_4', 'user_5', 'user_6'])
disp_mat = clicks_df.as_matrix(['displayed']).flatten()

def select_logged_event(t, k, d):
    # Extract all article vectors for first log item.
    _article_pool = eval(clicks_df.ix[t]['article_pool'])
    _article_vecs = np.array([articles_df.ix[x].values for x in _article_pool])
    # For each of the article row vectors, add a copy of the user features.
    logged = np.append(_article_vecs, values=np.tile(user_feat_mat[t], (len(_article_vecs), 1)), axis=1)
    rewards = np.array(list(map(lambda x : 1 if int(x) == disp_mat[t] else 0, _article_pool)))
    return logged.reshape((1, k, d)), rewards # (timestep, arm, feature)

In [9]:
def ols_intervals(X, Y, k, _delta, t, T):
    intervals = []
    for i in range(k):
        # Compute beta hat.
        _Xti = X[:t+1,i]
        _XtiT = transpose(_Xti)
        try:
            _XTX = pinv(_XtiT.dot(_Xti))
        except:
            print('Encountered singular matrix. Ignoring.')
            continue
        _Yti = Y[:t+1,i]
        Bh_t_i = _XTX.dot(_XtiT).dot(_Yti)  # Compute OLS estimators.
        yh_t_i = Bh_t_i.dot(X[t,i])
        _s2 = np.var(Y[:t+1,i])
        # Compute the confidence interval width using the inverse CDF.
        w_t_i = norm.ppf(1 - _delta/(2*T*k), loc=0, 
                         scale=np.sqrt(_s2 * X[t,i].dot(_XTX).dot(transpose(X[t,i]))))
        if not math.isnan(w_t_i):
            intervals.append([yh_t_i - w_t_i, yh_t_i + w_t_i])
    return intervals

In [10]:
def top_interval_yahoo(T):
    """
    Simulates T rounds of TopInterval for k.
    
    :param X: a 3-axis (T, k, d) ndarray of d-dimensional context vectors for each
              time-step and arm
    :param Y: a T x k ndarray of reward function output for each context vector
    :param k: the number of arms
    :param d: the number of features
    :param _delta: confidence parameter
    :param T: the number of iterations
    """
    k = 20
    d = 12
    X = np.empty((0, k, 12))
    Y = np.empty((0,20))
    _delta = 0.1
    
    _eta = eta(T)                               # exploration cutoff probabilities
    t = 0
    cursor = 0
#     for cursor in range(len(clicks_df)):
    for t in range(T):
#         if t >= T:
#             break
        print('Iteration [{0} / {1}]'.format(t, T))
        
        logged, rewards = select_logged_event(cursor, k, d)
        # Compute input vector for next item in stream and corresponding reward vector.
        Xp = np.vstack([X, logged])
        Yp = np.vstack([Y, rewards])
        
        if t <= d or np.random.rand() <= _eta[t]:
            # Play uniformly at random from [1, k].
#             print('Exploration round.')
            X, Y = Xp, Yp
            cursor += 1
        else:
            while (cursor <= len(clicks_df)):
#                 print('Exploitation round.')
                intervals = ols_intervals(Xp, Yp, k, _delta, t, T)
                # Pick the agent with the largest upper bound.
                pick = np.argmax(np.array(intervals)[:,1])
#                 print('Intervals: {0}'.format(intervals))
                _article_pool = eval(clicks_df.ix[cursor]['article_pool'])
#                 print('Pick: {0}. Displayed: {1}'.format(_article_pool[pick], clicks_df.ix[t]['displayed']))
                if clicks_df.ix[cursor]['displayed'] == int(_article_pool[pick]):
#                     print("Found match!")
                    X, Y = Xp, Yp
                    print('Iteration [{0} / {1}], matched with stream!'.format(t, T))
                    break
                cursor += 1
            
    # Compute sum of best picks over each iteration.
#     best = [Y[i].max() for i in range(2, T)]
#     performance = [Y[t][picks[t-2]] for t in range(2, T)]


In [11]:
tmp = top_interval_yahoo(1000)

Iteration [0 / 1000]
Iteration [1 / 1000]
Iteration [2 / 1000]
Iteration [3 / 1000]
Iteration [4 / 1000]
Iteration [5 / 1000]
Iteration [6 / 1000]
Iteration [7 / 1000]
Iteration [8 / 1000]
Iteration [9 / 1000]
Iteration [10 / 1000]
Iteration [11 / 1000]
Iteration [12 / 1000]
Iteration [13 / 1000]
Iteration [14 / 1000]


/Users/jtcho/Jupyter/cis625/fairml/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1907: RuntimeWarning: invalid value encountered in multiply
  lower_bound = self.a * scale + loc
/Users/jtcho/Jupyter/cis625/fairml/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1908: RuntimeWarning: invalid value encountered in multiply
  upper_bound = self.b * scale + loc


Iteration [14 / 1000], matched with stream!
Iteration [15 / 1000]
Iteration [15 / 1000], matched with stream!
Iteration [16 / 1000]
Iteration [17 / 1000]
Iteration [18 / 1000]
Iteration [18 / 1000], matched with stream!
Iteration [19 / 1000]
Iteration [19 / 1000], matched with stream!
Iteration [20 / 1000]
Iteration [20 / 1000], matched with stream!
Iteration [21 / 1000]
Iteration [22 / 1000]
Iteration [22 / 1000], matched with stream!
Iteration [23 / 1000]
Iteration [24 / 1000]
Iteration [25 / 1000]
Iteration [26 / 1000]
Iteration [26 / 1000], matched with stream!
Iteration [27 / 1000]
Iteration [28 / 1000]
Iteration [28 / 1000], matched with stream!
Iteration [29 / 1000]
Iteration [29 / 1000], matched with stream!
Iteration [30 / 1000]
Iteration [30 / 1000], matched with stream!
Iteration [31 / 1000]
Iteration [32 / 1000]
Iteration [32 / 1000], matched with stream!
Iteration [33 / 1000]
Iteration [33 / 1000], matched with stream!
Iteration [34 / 1000]
Iteration [34 / 1000], matched w

Iteration [160 / 1000]
Iteration [160 / 1000], matched with stream!
Iteration [161 / 1000]
Iteration [161 / 1000], matched with stream!
Iteration [162 / 1000]
Iteration [162 / 1000], matched with stream!
Iteration [163 / 1000]
Iteration [163 / 1000], matched with stream!
Iteration [164 / 1000]
Iteration [164 / 1000], matched with stream!
Iteration [165 / 1000]
Iteration [166 / 1000]
Iteration [166 / 1000], matched with stream!
Iteration [167 / 1000]
Iteration [167 / 1000], matched with stream!
Iteration [168 / 1000]
Iteration [168 / 1000], matched with stream!
Iteration [169 / 1000]
Iteration [169 / 1000], matched with stream!
Iteration [170 / 1000]
Iteration [170 / 1000], matched with stream!
Iteration [171 / 1000]
Iteration [171 / 1000], matched with stream!
Iteration [172 / 1000]
Iteration [172 / 1000], matched with stream!
Iteration [173 / 1000]
Iteration [173 / 1000], matched with stream!
Iteration [174 / 1000]
Iteration [174 / 1000], matched with stream!
Iteration [175 / 1000]
It

Iteration [302 / 1000], matched with stream!
Iteration [303 / 1000]
Iteration [303 / 1000], matched with stream!
Iteration [304 / 1000]
Iteration [305 / 1000]
Iteration [305 / 1000], matched with stream!
Iteration [306 / 1000]
Iteration [306 / 1000], matched with stream!
Iteration [307 / 1000]
Iteration [307 / 1000], matched with stream!
Iteration [308 / 1000]
Iteration [308 / 1000], matched with stream!
Iteration [309 / 1000]
Iteration [310 / 1000]
Iteration [310 / 1000], matched with stream!
Iteration [311 / 1000]
Iteration [312 / 1000]
Iteration [312 / 1000], matched with stream!
Iteration [313 / 1000]
Iteration [313 / 1000], matched with stream!
Iteration [314 / 1000]
Iteration [314 / 1000], matched with stream!
Iteration [315 / 1000]
Iteration [315 / 1000], matched with stream!
Iteration [316 / 1000]
Iteration [316 / 1000], matched with stream!
Iteration [317 / 1000]
Iteration [317 / 1000], matched with stream!
Iteration [318 / 1000]
Iteration [318 / 1000], matched with stream!
It

Iteration [447 / 1000], matched with stream!
Iteration [448 / 1000]
Iteration [448 / 1000], matched with stream!
Iteration [449 / 1000]
Iteration [449 / 1000], matched with stream!
Iteration [450 / 1000]
Iteration [450 / 1000], matched with stream!
Iteration [451 / 1000]
Iteration [451 / 1000], matched with stream!
Iteration [452 / 1000]
Iteration [452 / 1000], matched with stream!
Iteration [453 / 1000]
Iteration [453 / 1000], matched with stream!
Iteration [454 / 1000]
Iteration [454 / 1000], matched with stream!
Iteration [455 / 1000]
Iteration [455 / 1000], matched with stream!
Iteration [456 / 1000]
Iteration [457 / 1000]
Iteration [457 / 1000], matched with stream!
Iteration [458 / 1000]
Iteration [458 / 1000], matched with stream!
Iteration [459 / 1000]
Iteration [460 / 1000]
Iteration [460 / 1000], matched with stream!
Iteration [461 / 1000]
Iteration [461 / 1000], matched with stream!
Iteration [462 / 1000]
Iteration [462 / 1000], matched with stream!
Iteration [463 / 1000]
It

Iteration [585 / 1000], matched with stream!
Iteration [586 / 1000]
Iteration [586 / 1000], matched with stream!
Iteration [587 / 1000]
Iteration [587 / 1000], matched with stream!
Iteration [588 / 1000]
Iteration [588 / 1000], matched with stream!
Iteration [589 / 1000]
Iteration [589 / 1000], matched with stream!
Iteration [590 / 1000]
Iteration [590 / 1000], matched with stream!
Iteration [591 / 1000]
Iteration [591 / 1000], matched with stream!
Iteration [592 / 1000]
Iteration [592 / 1000], matched with stream!
Iteration [593 / 1000]
Iteration [593 / 1000], matched with stream!
Iteration [594 / 1000]
Iteration [594 / 1000], matched with stream!
Iteration [595 / 1000]
Iteration [596 / 1000]
Iteration [596 / 1000], matched with stream!
Iteration [597 / 1000]
Iteration [597 / 1000], matched with stream!
Iteration [598 / 1000]
Iteration [598 / 1000], matched with stream!
Iteration [599 / 1000]
Iteration [599 / 1000], matched with stream!
Iteration [600 / 1000]
Iteration [600 / 1000], m

ValueError: cannot reshape array of size 252 into shape (1,20,12)